In [75]:
import pandas as pd
import numpy as np

In [3]:
table = pd.read_pickle("./tmp/NL_experiment_results.pickle")

In [6]:
ARmodels = ["FRI", "SQ"] 

In [107]:
onlyAR = table[table.model.isin(ARmodels)]

onlyAR = onlyAR.set_index(["dataset","model"])

onlyAR_features = onlyAR[["features"]]

In [116]:
datasets = {
    "NL 1": {"n_features": 20, "n_strel": 10, "n_redundant": 0,},
    "NL 2": {"n_features": 20, "n_strel": 4, "n_redundant": 10,},
    "NL 3": {"n_features": 50, "n_strel": 10, "n_redundant": 10,},
    "NL 4": {"n_features": 80, "n_strel": 10, "n_redundant": 10},
}

In [133]:
from sklearn.metrics import precision_score, roc_auc_score, recall_score, f1_score

def get_truthAR(params):
    strong=params["n_strel"]
    weak=params["n_redundant"]
    irrel = params["n_features"] - strong - weak
    truth = [2] * strong + [1]*weak + [0] * irrel
    return truth


# We only take truth if relevance type (1 or 2) 
def get_truth_onetype(params, reltype):
    truth = get_truthAR(params)
    truth = np.array(truth)
    if reltype==0:
        raise Exception("0 makes no sense")
    return truth == reltype



def rowfunc(row, reltype, scorefunc):
    data = row.name[0]
    features = row["features"]
    pred = features == reltype

    truth = get_truth_onetype(datasets[data], reltype)
    if sum(truth)==0:
        return np.nan
    score  = scorefunc(truth, pred,zero_division=1)
    
    #if score == 0:
    #    score = np.nan
    return score

In [137]:
weakly_precision = onlyAR_features.apply(rowfunc, axis=1, args=[1,precision_score])
weakly_recall = onlyAR_features.apply(rowfunc, axis=1, args=[1,recall_score])
weakly = pd.concat([weakly_precision,weakly_recall],axis=1).rename(columns={0:"precision",1:"recall"})


strongly_precision = onlyAR_features.apply(rowfunc, axis=1, args=[2,precision_score])
strongly_recall = onlyAR_features.apply(rowfunc, axis=1, args=[2,recall_score])
strongly = pd.concat([strongly_precision,strongly_recall],axis=1).rename(columns={0:"precision",1:"recall"})
strongly

combined = pd.concat([weakly,strongly],axis=1,keys=["Weakly","Strongly"]).round(decimals=2)
combined

mean_datamodel = combined.groupby(["dataset","model"]).mean()

mean_model = mean_datamodel.groupby(["model"]).mean()

mean_model

precision  recall
dataset model                   
NL 1    FRI          NaN     NaN
        FRI          NaN     NaN
        FRI          NaN     NaN
        SQ           NaN     NaN
        SQ           NaN     NaN
        SQ           NaN     NaN
NL 2    FRI     0.909091     1.0
        FRI     0.909091     1.0
        FRI     0.909091     1.0
        SQ      1.000000     1.0
        SQ      0.000000     0.0
        SQ      1.000000     1.0
NL 3    FRI     0.555556     1.0
        FRI     0.588235     1.0
        FRI     0.588235     1.0
        SQ      0.642857     0.9
        SQ      0.857143     0.6
        SQ      0.769231     1.0
NL 4    FRI     0.714286     1.0
        FRI     0.714286     1.0
        FRI     0.769231     1.0
        SQ      0.434783     1.0
        SQ      0.434783     1.0
        SQ      0.434783     1.0